In [4]:
import tushare as ts
from typing import Any

import numpy as np
import pandas as pd

pro = ts.pro_api('2026c96ef5fa7fc3241c96baafd638c585284c7fefaa00b93ef0a62c')

def code_to_secid(df : pd.DataFrame , code_col = 'ts_code' , retain = False):
    '''switch old symbol into secid'''
    if code_col not in df.columns.values: return df
    replace_dict = {'T00018' : '600018'}
    df['secid'] = df[code_col].astype(str).str.slice(0, 6).replace(replace_dict)
    df['secid'] = df['secid'].where(df['secid'].str.isdigit() , '-1').astype(int)
    if not retain: del df[code_col]
    return df

# adj0 = pro.query('adj_factor',  trade_date='20240705')


In [17]:
date = 19970103
date_str = str(date)
adj = pro.query('adj_factor',  trade_date=date_str).rename(columns={'adj_factor':'adjfactor'})

quote = pro.daily(trade_date=date_str).rename(columns={'pct_chg':'pctchange','pre_close':'preclose','vol':'volume'})
quote['volume'] = quote['volume'] / 10. # to 10^3
quote['vwap'] = np.where(quote['volume'] == 0 , quote['close'] , quote['amount'] / quote['volume'])

shr = pro.daily_basic(trade_date=date_str).loc[:,['ts_code','trade_date' , 'total_share','float_share','free_share']]
shr.loc[:,['total_share','float_share','free_share']] *= 1e4
shr.loc[shr['free_share'].isna() , 'free_share'] = shr.loc[shr['free_share'].isna() , 'float_share']

limit = pro.stk_limit(trade_date=date_str)
if len(limit) == 0:
    limit = quote.loc[:,['ts_code' , 'trade_date' , 'close']].copy()
    limit['up_limit'] = (limit['close'] * 1.1).round(2)
    limit['down_limit'] = (limit['close'] * 0.9).round(2)
    limit = limit.drop(columns=['close'])

susp = pro.suspend_d(suspend_type='S', trade_date=date_str)

mutual_col = ['ts_code' , 'trade_date']

trade = quote.merge(adj,on=mutual_col,how='left').\
    merge(limit,on=mutual_col,how='left').\
    merge(shr,on=mutual_col,how='left')
trade['status'] = 1.0 * ~trade['ts_code'].isin(susp['ts_code']).fillna(0)
trade['limit'] = 1.0 * (trade['close'] >= trade['up_limit']).fillna(0) - 1.0 * (trade['close'] <= trade['down_limit']).fillna(0)
trade['turn_tt'] = (trade['volume'] / trade['total_share'] * 1e5).fillna(0)
trade['turn_fl'] = (trade['volume'] / trade['float_share'] * 1e5).fillna(0)
trade['turn_fr'] = (trade['volume'] / trade['free_share'] * 1e5).fillna(0)

trade = code_to_secid(trade).set_index('secid').sort_index().reset_index().loc[
    :,['secid', 'adjfactor', 'open', 'high', 'low', 'close', 'amount','volume', 'vwap', 
    'status', 'limit', 'pctchange', 'preclose', 'turn_tt','turn_fl', 'turn_fr']]
trade

,secid,adjfactor,open,high,low,close,amount,volume,vwap,status,limit,pctchange,preclose,turn_tt,turn_fl,turn_fr
0,1,13.968,16.50,16.59,16.18,16.30,118729.5778,7246.2,16.385082,1.0,0.0,-1.21,16.50,0.700411,1.014273,27.344151
1,2,5.121,10.60,11.20,10.38,10.45,66558.8782,6157.9,10.808697,1.0,0.0,0.00,10.45,1.942263,3.557413,21.992500
2,3,3.833,8.74,9.30,8.70,8.80,89765.3515,9964.3,9.008696,1.0,0.0,1.27,8.69,2.988391,5.196384,41.270681
3,4,3.112,6.15,6.20,5.87,6.00,9643.5748,1585.7,6.081588,1.0,0.0,-3.54,6.22,1.888262,3.806542,31.714000
4,5,2.275,9.96,10.30,9.60,9.70,44827.8666,4502.3,9.956659,1.0,0.0,-1.02,9.80,1.694824,3.574279,3.574279
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
511,600896,1.000,7.66,7.88,7.35,7.60,9195.5540,1203.1,7.643217,1.0,0.0,-0.78,7.66,0.721716,2.885132,2.885132
512,600897,1.000,17.40,17.40,16.80,16.84,12697.5670,746.7,17.004911,1.0,0.0,-3.22,17.40,0.691389,2.765556,2.765556
513,600898,1.320,8.00,8.05,7.47,7.60,11454.2910,1471.7,7.783034,1.0,0.0,-5.00,8.00,1.094004,2.198635,2.858225
514,600899,1.000,11.20,11.75,11.10,11.23,8995.8720,793.3,11.339811,1.0,0.0,0.45,11.18,0.717269,2.644333,2.644333


In [1]:
import tushare as ts
from src.data.tushare.basic import pro , code_to_secid , DateFetecher

import numpy as np
import pandas as pd

class MinQuote(DateFetecher):
    '''minute Quote'''
    def db_src(self): return 'trade_ts'
    def db_key(self): return 'min'    
    def get_data(self , date : int):
        date_str = str(date)
        adj = pro.query('adj_factor',  trade_date=date_str).rename(columns={'adj_factor':'adjfactor'})

        quote = pro.daily(trade_date=date_str).rename(columns={'pct_chg':'pctchange','pre_close':'preclose','vol':'volume'})
        quote['volume'] = quote['volume'] / 10. # to 10^3
        quote['vwap'] = np.where(quote['volume'] == 0 , quote['close'] , quote['amount'] / quote['volume'])

        shr = pro.daily_basic(trade_date=date_str).loc[:,['ts_code','trade_date' , 'total_share','float_share','free_share']]
        shr.loc[:,['total_share','float_share','free_share']] *= 1e4
        shr.loc[shr['free_share'].isna() , 'free_share'] = shr.loc[shr['free_share'].isna() , 'float_share']

        limit = pro.stk_limit(trade_date=date_str)
        if len(limit) == 0:
            limit = quote.loc[:,['ts_code' , 'trade_date' , 'close']].copy()
            limit['up_limit'] = (limit['close'] * 1.1).round(2)
            limit['down_limit'] = (limit['close'] * 0.9).round(2)
            limit = limit.drop(columns=['close'])

        susp = pro.suspend_d(suspend_type='S', trade_date=date_str)

        mutual_col = ['ts_code' , 'trade_date']

        trade = quote.merge(adj,on=mutual_col,how='left').\
            merge(limit,on=mutual_col,how='left').\
            merge(shr,on=mutual_col,how='left')
        trade['status'] = 1.0 * ~trade['ts_code'].isin(susp['ts_code']).fillna(0)
        trade['limit'] = 1.0 * (trade['close'] >= trade['up_limit']).fillna(0) - 1.0 * (trade['close'] <= trade['down_limit']).fillna(0)
        trade['turn_tt'] = (trade['volume'] / trade['total_share'] * 1e5).fillna(0)
        trade['turn_fl'] = (trade['volume'] / trade['float_share'] * 1e5).fillna(0)
        trade['turn_fr'] = (trade['volume'] / trade['free_share'] * 1e5).fillna(0)

        trade = code_to_secid(trade).set_index('secid').sort_index().reset_index().loc[
            :,['secid', 'adjfactor', 'open', 'high', 'low', 'close', 'amount','volume', 'vwap', 
            'status', 'limit', 'pctchange', 'preclose', 'turn_tt','turn_fl', 'turn_fr']]
        return trade
    
'''
df = ts.pro_bar(ts_code='600000.SH', api = pro , freq='1min', 
                start_date='2024-11-05 00:00:00', 
                end_date='2024-11-05 17:00:00')

df
'''


"\ndf = ts.pro_bar(ts_code='600000.SH', api = pro , freq='1min', \n                start_date='2024-11-05 00:00:00', \n                end_date='2024-11-05 17:00:00')\n\ndf\n"

In [56]:
import tushare as ts

from src.data.tushare.basic import pro
df = ts.pro_bar(api = pro , freq='1min', 
                start_date='2024-11-07 00:00:00', 
                end_date='2024-11-08 17:00:00')

df

抱歉，您每天最多访问该接口2次，权限的具体详情访问：https://tushare.pro/document/1?doc_id=108。
抱歉，您每天最多访问该接口2次，权限的具体详情访问：https://tushare.pro/document/1?doc_id=108。
抱歉，您每分钟最多访问该接口2次，权限的具体详情访问：https://tushare.pro/document/1?doc_id=108。


OSError: ERROR.

In [12]:
pro.daily(start_date='20240708')

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
0,003032.SZ,20240708,9.47,9.50,8.74,8.88,9.47,-0.59,-6.2302,195958.79,177127.325
1,002688.SZ,20240708,3.68,3.70,3.59,3.60,3.71,-0.11,-2.9650,66932.00,24218.867
2,688238.SH,20240708,4.56,4.56,4.30,4.32,4.56,-0.24,-5.2632,54944.70,24271.305
3,605090.SH,20240708,28.76,28.86,28.07,28.45,28.87,-0.42,-1.4548,43069.40,122560.860
4,600611.SH,20240708,2.77,3.04,2.77,2.95,2.78,0.17,6.1151,466175.26,135351.973
...,...,...,...,...,...,...,...,...,...,...,...
5328,002873.SZ,20240708,8.58,8.63,8.21,8.23,8.53,-0.30,-3.5170,26403.99,21990.333
5329,001373.SZ,20240708,23.85,24.15,23.12,23.26,23.91,-0.65,-2.7185,6656.00,15626.912
5330,603018.SH,20240708,8.52,8.55,8.25,8.35,8.56,-0.21,-2.4533,171655.07,143505.269
5331,000797.SZ,20240708,2.35,2.36,2.27,2.28,2.37,-0.09,-3.7975,176813.00,40648.298
